# Leveraging Linguistics


We are going to pick up a simple use case and see how we can solve that. Then, we repeat this again, but on a slighlty different text corpus and so on. 

This helps us learn build intuition on how to use linguistics in NLP. As mentioned, I am going to use spaCy here, but you are free to use NLTK or anything else available in your favourite progamming language

## Grammar Crash Course and spaCy

In [1]:
# !python -m spacy download en_core_web_lg

In [2]:
import spacy
from spacy import displacy # for visualization
nlp = spacy.load('en_core_web_lg')

In [3]:
import textacy

If there is an error above, try:
- Windows Shell:```python -m spacy download en``` as **Administrator**
- Linux Terminal:```sudo python -m spacy download en ```

The first half talks about NLP pipeline 

## Redacting Names with Named Entity Recognition

In [4]:
text = "Madam Pomfrey, the nurse, was kept busy by a sudden spate of colds among the staff and students. Her Pepperup potion worked instantly, though it left the drinker smoking at the ears for several hours afterward. Ginny Weasley, who had been looking pale, was bullied into taking some by Percy."

In [5]:
# Parse the text with spaCy. This runs the entire pipeline.
doc = nlp(text)

In [6]:
# 'doc' now contains a parsed version of text. We can use it to do anything we want!
# For example, this will print out all the named entities that were detected:
for entity in doc.ents:
    print(f"{entity.text} ({entity.label_})")

Pomfrey (PERSON)
Pepperup (ORG)
several hours (TIME)
Ginny Weasley (PERSON)
Percy (PERSON)


#TODO: Explain the entity and entity labels above

In [7]:
def redact_names(text):
    doc = nlp(text)
    redacted_sentence = []
    for token in doc:
        if token.ent_type_ == "PERSON":
            redacted_sentence.append("[REDACTED]")
        else:
            redacted_sentence.append(token.string)
    return "".join(redacted_sentence)

In [8]:
redact_names(text)

'Madam [REDACTED], the nurse, was kept busy by a sudden spate of colds among the staff and students. Her Pepperup potion worked instantly, though it left the drinker smoking at the ears for several hours afterward. [REDACTED][REDACTED], who had been looking pale, was bullied into taking some by [REDACTED].'

In [9]:
def redact_names(text):
    doc = nlp(text)
    redacted_sentence = []
    for ent in doc.ents:
        ent.merge()
    for token in doc:
        if token.ent_type_ == "PERSON":
            redacted_sentence.append("[REDACTED]")
        else:
            redacted_sentence.append(token.string)
    return "".join(redacted_sentence)

In [10]:
redact_names(text)

'Madam [REDACTED], the nurse, was kept busy by a sudden spate of colds among the staff and students. Her Pepperup potion worked instantly, though it left the drinker smoking at the ears for several hours afterward. [REDACTED], who had been looking pale, was bullied into taking some by [REDACTED].'

## Entity Types 

In [11]:
def explain_text_entities(text):
    doc = nlp(text)
    for ent in doc.ents:
        print(f'{ent}, Label: {ent.label_}, {spacy.explain(ent.label_)}')

In [12]:
explain_text_entities('Tesla has gained 20% market share in the months since')

Tesla, Label: ORG, Companies, agencies, institutions, etc.
20%, Label: PERCENT, Percentage, including "%"
the months, Label: DATE, Absolute or relative dates or periods


In [13]:
explain_text_entities('Taj Mahal built by Mughal Emperor Shah Jahan stands tall on the banks of Yamuna in modern day Agra, India')

Taj Mahal, Label: PERSON, People, including fictional
Mughal, Label: NORP, Nationalities or religious or political groups
Shah Jahan, Label: PERSON, People, including fictional
Yamuna, Label: LOC, Non-GPE locations, mountain ranges, bodies of water
Agra, Label: GPE, Countries, cities, states
India, Label: GPE, Countries, cities, states


In [14]:
explain_text_entities('Ashoka was a great Indian king')

Ashoka, Label: PERSON, People, including fictional
Indian, Label: NORP, Nationalities or religious or political groups


In [15]:
explain_text_entities('The Ashoka University sponsors the Young India Fellowship')

Ashoka University, Label: ORG, Companies, agencies, institutions, etc.
the Young India Fellowship, Label: ORG, Companies, agencies, institutions, etc.


# Question Generation with PoS Tagging

Sometimes, we want to quickly pull out keywords, or keyphrases from a larger body of text. This helps us mentally paint a picture of what this text is about. This is particularly helpful in analysis of texts like long emails or essays. 

We refer to these as noun chunks. Noun chunks are _noun phrases_ - not a single word, but a short phrase which describes the noun. For example, "the blue skies" or "the world’s largest conglomerate". 

To get the noun chunks in a document, simply iterate over `doc.noun_chunks`: 

In [16]:
example_text = 'Bansoori is an Indian classical instrument. Tom plays Bansoori and Guitar.'

In [17]:
doc = nlp(example_text)

In [18]:
for idx, sentence in enumerate(doc.sents):
    for noun in sentence.noun_chunks:
        print(f'sentence{idx+1}', noun)

sentence1 Bansoori
sentence1 an Indian classical instrument
sentence2 Tom
sentence2 Bansoori
sentence2 Guitar


Our example text has two sentences, we can pull out noun phrase chunks from each sentence. We pull out noun phrases instead of single words. This means, we are able to pull out 'an Indian classical instrument' as one noun. This is quite useful as we will see in a moment.  

Next, let's take a quick look at all parts-of-speech tags in our example text. We will use the verbs and adjectives to write some simple question generating logic. 

In [19]:
for token in doc:
    print(token, token.pos_, token.tag_)

Bansoori PROPN NNP
is VERB VBZ
an DET DT
Indian ADJ JJ
classical ADJ JJ
instrument NOUN NN
. PUNCT .
Tom PROPN NNP
plays VERB VBZ
Bansoori PROPN NNP
and CCONJ CC
Guitar PROPN NNP
. PUNCT .


Notice that here 'instrument' is tagged as a NOUN while 'Indian' and 'classical' are tagged as adjectives. This makes sense. Addititionally, Bansoor and Guitar are tagged as PROPN or Proper Nouns. 

**Nouns vs Proper Noun** 
Nouns name people, places, and things. Common nouns name general items like waiter, jeans, country. Proper nouns name specific things like Roger, Levi's, India

In [20]:
ruleset = [
    {
        'id': 1, 
        'req_tags': ['NNP', 'VBZ', 'NN'],
    }, 
    {
        'id': 2, 
        'req_tags': ['NNP', 'VBZ'],
    }
    ]

In [21]:
print(ruleset)

[{'id': 1, 'req_tags': ['NNP', 'VBZ', 'NN']}, {'id': 2, 'req_tags': ['NNP', 'VBZ']}]


In [22]:
def get_pos_tag(doc, tag):
    return [tok for tok in doc if tok.tag_ == tag]

In [23]:
def sent_to_ques(sent):
    doc = nlp(sent)
    pos_tags = [token.tag_ for token in doc]
    for idx, rule in enumerate(ruleset):
        if rule['id'] == 1:
            if all(key in pos_tags for key in rule['req_tags']): 
                print(sent)
                NNP = get_pos_tag(doc, "NNP")
                NNP = str(NNP[0])
                VBZ = get_pos_tag(doc, "VBZ")
#                 subjects = get_subjects_of_verb(VBZ[0])
#                 print(subjects)
                VBZ = str(VBZ[0])
                ques = f'What {VBZ} {NNP}?'
                return(ques)
        if rule['id'] == 2:
            if all(key in pos_tags for key in rule['req_tags']): #'NNP', 'VBZ' in sentence.
                print(sent)
                NNP = get_pos_tag(doc, "NNP")
                NNP = str(NNP[0])
                VBZ = get_pos_tag(doc, "VBZ")
                VBZ = str(VBZ[0].lemma_)
                ques = f'What does {NNP} {VBZ}?'
                return (ques)

for sent in doc.sents:
    print(sent_to_ques(str(sent)))

Bansoori is an Indian classical instrument.
What is Bansoori?
Tom plays Bansoori and Guitar.
What does Tom play?


# Question Generation using Dependency Parsing

What is a dependency parser? 

> A dependency parser analyzes the grammatical structure of a sentence, establishing relationships between "head" words and words which modify those heads.
> from [Stanford NNDEP Project](https://nlp.stanford.edu/software/nndep.html)

A dependency parser helps us understand the various ways in which parts of the sentence interact or depend on each other. For instance, how is the noun modified by adjectives. 

In [24]:
for token in doc:
    print(token, token.dep_)

Bansoori nsubj
is ROOT
an det
Indian amod
classical amod
instrument attr
. punct
Tom nsubj
plays ROOT
Bansoori dobj
and cc
Guitar conj
. punct


Some of these terms are simple enough to guess e.g. 'ROOT' is where the dependency tree might begin. 'nsubj' is the noun or nominal subject. 'cc' is probably conjunction. But this is still incomplete, luckily for us, spaCy includes the nifty `explain()` function to help us interpret these.  

In [25]:
for token in doc:
    print(token, token.dep_, spacy.explain(token.dep_))

Bansoori nsubj nominal subject
is ROOT None
an det determiner
Indian amod adjectival modifier
classical amod adjectival modifier
instrument attr attribute
. punct punctuation
Tom nsubj nominal subject
plays ROOT None
Bansoori dobj direct object
and cc coordinating conjunction
Guitar conj conjunct
. punct punctuation


This gives us a good starting point to Google away and pick up some linguistics specific terms. E.g. a 'conjunct' is often used to connect two clauses. 'attribute' is simply a way to highlight something which is a property of the nominal subject. 

Nominal subjects are usually nouns or pronouns which in turns are actors (via verbs) or have properties(via attributes). 

## Visualizing the Relationship

spaCy has an inbuilt tool called displacy for displaying simple, but clean and powerful visualizations. It offers two primary modes: Named Entity Recognition and Dependency Parsing. Here we will use the 'dep' or dependency mode. 

In [26]:
displacy.render(doc, style='dep', minify=True, jupyter=True)

Let's take the first sentence for a quick study: We see that "instrument" is "amod" or adjectively modified by "Indian classicial". We pulled this phrase earlier as a noun chunk. 

This means that when we pulled noun phrase chunks out of this sentence, spaCy must have finished dependency parsing already under the hood. 

Also notice the direction of arrows, while the NOUN (instrument) is modified by ADJ. It is the 'attr' of the ROOT VERB (is). 


This logical tree structure of simple sentences is what we will exploit to simplify our question generation. Let's write some functions to extract these dependency entities in the spaCy token format i.e. without converting them to strings. 

In [27]:
def get_main_verbs_of_sent(sent):
    """Return the main (non-auxiliary) verbs in a sentence."""
    return [tok for tok in sent
            if tok.pos_ == 'VERB' and tok.dep_ not in {'aux', 'auxpass'}]

If we are to ask questions from someone, they often are around a piece of information e.g. What is the capital of India? or around some action e.g. What did you do on Sunday?

Of course, there are other types of questions around Who, Where, Why and so on. 

Extend the approach below to at least add Who, Where and When questions as practice. 

In [28]:
def _get_conjuncts(tok):
    """
    Return conjunct dependents of the leftmost conjunct in a coordinated phrase,
    e.g. "Burton, [Dan], and [Josh] ...".
    """
    return [right for right in tok.rights
            if right.dep_ == 'conj']

In [29]:
def get_subjects_of_verb(verb):
    SUBJ_DEPS = {'agent', 'csubj', 'csubjpass', 'expl', 'nsubj', 'nsubjpass'}
    """Return all subjects of a verb according to the dependency parse."""
    subjs = [tok for tok in verb.lefts
             if tok.dep_ in SUBJ_DEPS]
    # get additional conjunct subjects
    subjs.extend(tok for subj in subjs for tok in _get_conjuncts(subj))
    return subjs

# Level Up: Question and Answer
So far, we have been trying to generate questions. But if you were trying to make an automated quiz for students, you would also need to mine the right answer. 

The answer in this case will be simply the objects of verb. What is an object of verb? 

> In the sentence, "Give the book to me," "book" is the direct object of the verb "give," and "me" is the indirect object. - from the Cambridge English Dictionary

Loosely, object is the piece on which our verb acts. This is almost always the answer to our "what". Let's write a question to find the objects of any verb: 

In [30]:
def get_objects_of_verb(verb):
    """
    Return all objects of a verb according to the dependency parse,
    including open clausal complements.
    """
    OBJ_DEPS = {'attr', 'dobj', 'dative', 'oprd'}
    objs = [tok for tok in verb.rights
            if tok.dep_ in OBJ_DEPS]
    # get open clausal complements (xcomp)
    objs.extend(tok for tok in verb.rights
                if tok.dep_ == 'xcomp')
    # get additional conjunct objects
    objs.extend(tok for obj in objs for tok in _get_conjuncts(obj))
    return objs

Let's look at the output of our functions for the example text. The first is the sentence itself, then the root verb, than the lemma form of that verb, followed by subjects of the verb and then objects.  

In [31]:
doc = nlp(example_text)
for sentence in doc.sents:
    print(sentence, sentence.root, sentence.root.lemma_, get_subjects_of_verb(sentence.root), get_objects_of_verb(sentence.root))

Bansoori is an Indian classical instrument. is be [Bansoori] [instrument]
Tom plays Bansoori and Guitar. plays play [Tom] [Bansoori, Guitar]


Let's arrange the pieces above into a neat function which we can then re-use

In [32]:
def para_to_ques(eg_text):
    doc = nlp(eg_text)
    results = []
    for sentence in doc.sents:
        root = sentence.root
        ask_about = get_subjects_of_verb(root)
        answers = get_objects_of_verb(root)
        if len(ask_about) > 0 and len(answers) > 0:
            if root.lemma_ == "be":
                question = f'What {root} {ask_about[0]}?'
            else:
                question = f'What does {ask_about[0]} {root.lemma_}?'
            results.append({'question':question, 'answers':answers})
    return results
para_to_ques(example_text)

[{'question': 'What is Bansoori?', 'answers': [instrument]},
 {'question': 'What does Tom play?', 'answers': [Bansoori, Guitar]}]

This seems right to me. Let's run this on a larger sample of sentences. This sample has varying degrees of complexities and sentence structures. 

In [33]:
large_example_text = """
Puliyogare is a South Indian dish made of rice and tamarind. 
Priya writes poems. Shivangi bakes cakes. Sachin sings in the orchestra.

Osmosis is the movement of a solvent across a semipermeable membrane toward a higher concentration of solute. In biological systems, the solvent is typically water, but osmosis can occur in other liquids, supercritical liquids, and even gases.
When a cell is submerged in water, the water molecules pass through the cell membrane from an area of low solute concentration to high solute concentration. For example, if the cell is submerged in saltwater, water molecules move out of the cell. If a cell is submerged in freshwater, water molecules move into the cell.

Raja-Yoga is divided into eight steps. The first is Yama. Yama is nonviolence, truthfulness, continence, and non-receiving of any gifts.
After Yama, Raja-Yoga has Niyama. cleanliness, contentment, austerity, study, and self - surrender to God.
The steps are Yama and Niyama. 
"""


In [34]:
para_to_ques(large_example_text)

[{'question': 'What is Puliyogare?', 'answers': [dish]},
 {'question': 'What does Priya write?', 'answers': [poems]},
 {'question': 'What does Shivangi bake?', 'answers': [cakes]},
 {'question': 'What is Osmosis?', 'answers': [movement]},
 {'question': 'What is solvent?', 'answers': [water]},
 {'question': 'What is first?', 'answers': [Yama]},
 {'question': 'What is Yama?',
  'answers': [nonviolence, truthfulness, continence, of]},
 {'question': 'What does Yoga have?', 'answers': [Niyama]},
 {'question': 'What are steps?', 'answers': [Yama, Niyama]}]

# Facts Extraction using Semi Structured Sentence Parsing
Introducing textacy,

Boss mode with co reference resolution